In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

model_name='Llama2-7b-hf'
#model_name='Llama-3.2-1B'

DTYPE = torch.float16
fp_model, _ = qlib.load_llama(
	model_name=model_name, 
	device_map='cpu', 
	torch_dtype=DTYPE
)

qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
	f'/home/msst/repo/Quantization/ml/llm/pretrained_models/Llama2-7b-hf',
	torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LOL, Im custom!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of QuantizedLlamaForCausalLM were not initialized from the model checkpoint at /home/msst/repo/Quantization/ml/llm/pretrained_models/Llama2-7b-hf and are newly initialized: ['model.layers.0.mlp.down_proj.SU', 'model.layers.0.mlp.down_proj.SV', 'model.layers.0.mlp.down_proj.scales', 'model.layers.0.mlp.down_proj.trellis', 'model.layers.0.mlp.down_proj.weight_quantizer.lut', 'model.layers.0.mlp.down_proj.weight_quantizer.tlut', 'model.layers.0.mlp.gate_proj.SU', 'model.layers.0.mlp.gate_proj.SV', 'model.layers.0.mlp.gate_proj.scales', 'model.layers.0.mlp.gate_proj.trellis', 'model.layers.0.mlp.gate_proj.weight_quantizer.lut', 'model.layers.0.mlp.gate_proj.weight_quantizer.tlut', 'model.layers.0.mlp.up_proj.SU', 'model.layers.0.mlp.up_proj.SV', 'model.layers.0.mlp.up_proj.scales', 'model.layers.0.mlp.up_proj.trellis', 'model.layers.0.mlp.up_proj.weight_quantizer.lut', 'model.layers.0.mlp.up_proj.weight_quantizer.tlut', 'model.layers.0.self_attn.k_proj.SU', 'model.layers.0.sel

In [2]:
for module_name, qmodule in qmodel.named_modules():
    if isinstance(qmodule, qlib.TrellisLinear):
        qmodule.wrap_module(
            fp_model.get_submodule(module_name), 
            verbose=True,
            module_name=module_name)

# quantlut_sym  
# self_attn.q_proj: 0.074
# self_attn.k_proj: 0.073
# self_attn.v_proj: 0.071
# self_attn.o_proj: 0.071
# mlp.gate_proj:    0.071

# Rand2d  
# self_attn.q_proj: 0.072
# self_attn.k_proj: 0.071
# self_attn.v_proj: 0.069
# self_attn.o_proj: 0.069
# mlp.gate_proj:    0.069

# LowBitSym 4bit = 5bit
# self_attn.q_proj: 0.073
# self_attn.k_proj: 0.072
# self_attn.v_proj: 0.070
# self_attn.o_proj: 0.070
# mlp.gate_proj:    0.070


model.layers.0.self_attn.q_proj


100%|██████████| 256/256 [00:19<00:00, 13.01it/s]


error (w-wq)^2: 0.073
model.layers.0.self_attn.k_proj


100%|██████████| 256/256 [00:19<00:00, 12.99it/s]


error (w-wq)^2: 0.072
model.layers.0.self_attn.v_proj


...

error (w-wq)^2: 0.070
model.layers.31.mlp.up_proj


100%|██████████| 688/688 [00:53<00:00, 12.93it/s]


error (w-wq)^2: 0.070
model.layers.31.mlp.down_proj


100%|██████████| 688/688 [00:53<00:00, 12.93it/s]


error (w-wq)^2: 0.070


In [3]:
tlp = qmodel.config.trellis_linear_params
checkpoints_path = '/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/'

qmodel.save_pretrained(
	os.path.join(
		checkpoints_path, 
	    'T{T}_L{L}_V{V}_K{K}_cbs{tlut_bits}_{decode_mode}_{incoh_proc_mode}'.format(
			T=tlp['T'],
			L=tlp['L'],
			V=tlp['V'],
			K=tlp['K'],
			tlut_bits=tlp['tlut_bits'],
			decode_mode=tlp['decode_mode'],
			incoh_proc_mode=tlp['incoh_proc_mode'])
	)
)